In [10]:
pip install Pillow PyMuPDF openai

Using cached pytesseract-0.3.10-py3-none-any.whl (14 kB)
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.2 MB 330.3 kB/s eta 0:00:10
   - -------------------------------------- 0.1/3.2 MB 939.4 kB/s eta 0:00:04
   --- ------------------------------------ 0.3/3.2 MB 1.8 MB/s eta 0:00:02
   ---- ----------------------------------- 0.4/3.2 MB 2.2 MB/s eta 0:00:02
   ----- ---------------------------------- 0.5/3.2 MB 2.0 MB/s eta 0:00:02
   ----- ---------------------------------- 0.5/3.2 MB 2.0 MB/s eta 0:00:02
   ----- ---------------------------------- 0.5/3.2 MB 1.6 MB/s eta 0:00:02
   --------- ------------------------------ 0.7/3.2 MB 2.1 MB/s eta 0:00:02
   ---------- ----------------------------- 0.9/3.2 MB 2.2 MB/s eta 0:00:02
   ------------ --------------------------- 1.0/3.2 MB 2.3 MB/s eta 0:00:01
   -------------- -----------------------

# Process pdf file

In [1]:
import fitz
from PIL import Image
import io

def pdf_to_images(pdf_path, zoom_x=2.0, zoom_y=2.0):
    pdf_document = fitz.open(pdf_path)
    images = []
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        mat = fitz.Matrix(zoom_x, zoom_y)
        pix = page.get_pixmap(matrix=mat)
        img = Image.open(io.BytesIO(pix.tobytes("png")))
        images.append(img)
    return images

pdf_path = 'thuvienphapluat/14_nghị_định_42_2023_hướng_dẫn_điều_chỉnh_lương_hưu,_trợ_cấp_BHXH.pdf'
images = pdf_to_images(pdf_path, 2.5, 2.5)

In [2]:
len(images)

6

In [3]:
import base64
from io import BytesIO
image_base64 = []

for image in images:
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    image_bytes = buffered.getvalue()
    image_base64.append(base64.b64encode(image_bytes).decode("utf-8"))



In [4]:
len(image_base64)

6

In [27]:
contents = []
pages = []

# Parse text

In [7]:
prompt_system = """You are an AI assistant that processes images containing legal content. Your task is to transcribe the content from the image into markdown format with the following rules:
        
        1. Use `#` for titles if you see the name of a Law or Decree.
        2. Use `##` if you see the word "Chương" or "Mẫu số" at the beginning of a sentence.
        3. Use `###` if you see the word "Mục" at the beginning of a sentence.
        4. Use `####` if you see the word "Điều" at the beginning of a sentence.
        5. Do not use any '#' for a), b), c),... etc or 1), 2), 3),...etc
        6. If you encounter a equation, notice the following rules:
           - The equation can be on multiple lines, detect all the lines of the equation.
           - Convert the method in the image into a single latex block with all expressions inside a pair of dollar signs. Try to get both legs before and after the equal sign ($$) (i.e. $$ \text{Gross bonus amount} = \text{Bonus} \times \left(100 - \frac{\text{Date of absent in a year}}{22 \times 12}\right) $$)
        7. Only respond with the markdown content, without any additional explanation or description.
        8. Don't annotate ```markdown\n  in the response
"""

In [29]:
import base64
import requests
from io import BytesIO
import os
import time
# OpenAI API Key
api_key = os.environ["OPENAI_API_KEY"]

# # Function to encode the image
# def encode_image(image_path):
#   with open(image_path, "rb") as image_file:
#     return base64.b64encode(image_file.read()).decode('utf-8')

# # Path to your image
# image_path = "E:/LLM_Coursse/thuvienphapluat/Screenshot 2024-08-03 084051.png"

# # Getting the base64 string
# image_base64 = encode_image(image_path)
def parse_pdf():
  for i in range(0, len(image_base64)):
    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    }

    payload = {
      "model": "gpt-4o-mini",
      "messages": [
        {
          "role": "system",
          "content": [
            {
              "type": "text",
              "text": prompt_system
            },
          ]
        },
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": "Transcribe the content from this image into markdown format"
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{image_base64[i]}"
              }
            }
          ]
        }
      ],
      "max_tokens": 4096
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    print(f"{i}:")
    print(response.json())

    # print(response.json())
    contents.append(response.json()['choices'][0]['message']['content'])
    pages.append(response.json()['choices'][0]['message']['content'])
    time.sleep(2)
  

In [30]:
parse_pdf()

0:
{'id': 'chatcmpl-A0Oc8hLNSHy5cXwk8W8ksUna8FQ9x', 'object': 'chat.completion', 'created': 1724657980, 'model': 'gpt-4o-mini-2024-07-18', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '# NGHỊ ĐỊNH\nĐiều chỉnh lương hưu, trợ cấp bảo hiểm xã hội và trợ cấp hằng tháng\n\nCăn cứ Luật Tổ chức Chính phủ ngày 19 tháng 6 năm 2015; Luật sửa đổi, bổ sung một số điều của Luật Tổ chức Chính phủ và Luật Tổ chức chính quyền địa phương ngày 22 tháng 11 năm 2019;\n\nCăn cứ Luật Bảo hiểm xã hội ngày 20 tháng 11 năm 2014;\n\nCăn cứ Luật An toàn, vệ sinh lao động ngày 25 tháng 6 năm 2015;\n\nCăn cứ Nghị quyết số 69/2022/QH15 ngày 11 tháng 11 năm 2022 của Quốc hội về dự toán ngân sách nhà nước năm 2023;\n\nTheo đề nghị của Bộ trưởng Bộ Lao động - Thương binh và Xã hội;\n\nChính phủ ban hành Nghị định điều chỉnh lương hưu, trợ cấp bảo hiểm xã hội và trợ cấp hằng tháng.\n\n## Chương I\nQUY ĐỊNH CHUNG\n\n#### Điều 1. Đối tượng điều chỉnh\n1. Nghị định này điều chỉnh mức lương hưu, trợ

In [11]:
file_name = os.path.basename(pdf_path)
file_name_without_extension = os.path.splitext(file_name)[0]
file_name_without_extension

'8_Nghị_định_15_2020_về_lao_động_nước_ngoài_tại_VN'

In [10]:
len(pages)

29

In [13]:
for i in range(0, len(pages)):
    with open(f"E:/LLM_Coursse/thuvienphapluat/parsed_file/{file_name_without_extension}.md", "a", encoding="utf-8") as file:
        file.write("\n" + f"# Trang {i+1}")
        file.write("\n" + pages[i])

# Parse table and fomula

In [52]:
column_tree = """
1. STT
2. Họ tên
3. Mã số BHXH
4. Ngày tháng năm sinh
5. Giới tính
6. Số CCCD/CMND/Hộ chiếu
7. Cấp bậc, chức vụ, chức danh nghề, nơi làm việc
8. Vị trí việc làm
   - Nhà quản lý
   - Chuyên môn kỹ thuật bậc cao
   - Chuyên môn kỹ thuật bậc trung
   - Khác
9. Tiền lương
   - Hệ số/Bậc lương
   - Phụ cấp
     - Chức vụ
     - Thâm niên VK
     - Lưu động
     - Thu hút
     - Ngành nghề
     - Khu vực
     - Khoán
     - Phụ cấp lương bổ sung
10. Ngành/nghề nặng nhọc, độc hại
11. Loại và hiệu lực hợp đồng lao động
    - Hiệu lực HĐLD xác định thời hạn
    - Hiệu lực HĐLD không xác định thời hạn
    - Hiệu lực HĐLD khác (dưới 1 tháng, thử việc)
    - Ngày bắt đầu HĐLD
    - Ngày kết thúc HĐLD
12. Thời điểm đơn vị bắt đầu đóng BHXH
13. Thời điểm đơn vị kết thúc đóng BHXH
14. Ghi chú
"""

json_c = """
{
    "table": [
        {
            "STT": "",
            "Họ tên": "",
            "Mã số BHXH": "",
            "Ngày tháng năm sinh": "",
            "Giới tính": "",
            "Số CCCD/CMND/ Hộ chiếu": "",
            "Cấp bậc, chức vụ, chức danh nghề, nơi làm việc": "",
            "Vị trí việc làm": {
                "Nhà quản lý": "",
                "Chuyên môn kỹ thuật bậc cao": "",
                "Chuyên môn kỹ thuật bậc trung": "",
                "Khác": ""
            },
            "Tiền lương": {
                "Hệ số lương": "",
                "Chức vụ": "",
                "Thâm niên VK": "",
                "Thâm niên nghề": "",
                "Phụ cấp khác": ""
            },
            "Ngành/nghề nặng nhọc, độc hại": "",
            "Loại và hiệu lực hợp đồng lao động": {
                "Ngày bắt đầu HĐLĐ không xác định thời hạn": "",
                "Ngày kết thúc": "",
                "Ngày bắt đầu HĐLĐ xác định thời hạn": "",
                "Ngày kết thúc": "",
                "Hiệu lực HĐLĐ khác (dưới 1 tháng, thử việc)": ""
            },
            "Thời điểm đơn vị bắt đầu đóng BHXH": "",
            "Thời điểm đơn vị kết thúc đóng BHXH": "",
            "Ghi chú": ""
        }
    ]
}
"""

list_columns ="""
List all the columns of the table, if the table has nested columns, represent in a hierarchical tree
"""

build_tree = """
Build the hierarchical trees of the columns of the table in the image without any additional explanation or description.
"""

list_rows = """
List all the rows'names of table in the image (if has any rows have name)
"""

construct_json_prompt = """
You are assitant in parsing image to json. Given a image, parse it to json with the following rules:
  1. Only respond with the json content, without any additional explanation or description. 
  2. Do not annotate ```json in response.
"""

user_json_prompt = """
Parse the table in this image to json
"""

construct_fomula_prompt = """
You are assitant in parsing image to markdown. Given a image of a fomula, parse it to markdown with the following rules:
  1. Only respond with the markdown content, without any additional explanation or description. 
  2. Do not annotate ```markdown in response.
"""

fomula_prompt = """Parse the formula in the image to a single markdown block with all expressions inside one pair of double dollar signs ($$)."""

reconstruct_table_prompt = """
You are assitant in parsing image to markdown. Given a image, the hierarchical trees of the columns of the table in the image and json of the table, parse it to markdown with the following rules:
  1. Only respond with the markdown content, without any additional explanation or description. 
  2. Do not annotate ```markdown in response.
"""

final_prompt = f"Here is the hierarchical trees of the columns of the table in the image: {column_tree} and json of the table: {json_c}. Parse the following image to markdown"

reconstruct_table_prompt2 = """
You are assitant in parsing image to markdown. Given a image, the hierarchical trees of the columns of the table in the image, parse it to markdown with the following rules:
  1. Only respond with the markdown content, without any additional explanation or description. 
  2. Do not annotate ```markdown in response.
"""

final_prompt2 = f"Here is the hierarchical trees of the columns of the table in the image: {column_tree}. Parse the following image to markdown"
latex_2_markdown = """Chuyển công thức latex về markdown. Ví dụ về dạng markdown của công thức như sau:
          Midnight Shift Allowance (per 1 hour) = 
[(Basic Salary + Allowance relative to working) * 0.3] 
/ 
(Standard Monthly Working Days * Standard Working Hours per day)

= [(Basic Salary + Allowance relative to working) * 0.3] / [22 (days) * 8 (hours)]"""
detect_table = """You are an expert in table dectection. If the input image have any tables,focus on each table and response the markdown of each table with the index of each table in the begin of table (i.e. Table 1, Table 2, ...etc). Try to keep all the text of each table at the right cell in the image and do not annotate ```markdown. If the image don't have any tables, response "No" """

In [8]:
import base64
import requests
from io import BytesIO
import os
api_key = os.environ["OPENAI_API_KEY"]
headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}
from PIL import Image
import base64
import io

def resize_image(image_path, scale_factor):
    img = Image.open(image_path)

    width, height = img.size

    new_width = int(width * scale_factor)
    new_height = int(height * scale_factor)

    img_resized = img.resize((new_width, new_height))
    
    return img_resized

def encode_image(image):
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

image_path = "table/Screenshot 2024-09-07 173405.png"
resized_image = resize_image(image_path, 2)
image_base64 = encode_image(resized_image)

payload = {
  "model": "gpt-4o-mini",
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": prompt_system
        },
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": """Parse this image to markdown"""
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{image_base64}"
          }
        }
      ]
    }
  ],
  "max_tokens": 4096
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-A52jH02FdhKipwrkc7Zhx8smphLhK', 'object': 'chat.completion', 'created': 1725765495, 'model': 'gpt-4o-mini-2024-07-18', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '# hợp sau đây:\na. Thuế thu nhập cá nhân  \nb. Bảo hiểm xã hội, bảo hiểm y tế, bảo hiểm thất nghiệp  \nc. Các khoản trừ trước  \nd. Bồi thường  \nd. Các chi phí khác do Công ty và Công đoàn thoả thuận.  \n\n## 2) Trường hợp ngày thanh toán lương là ngày nghỉ lễ thì việc trả lương được thực hiện vào ngày làm việc trước đó.  \n## 3) Tiền lương được ưu tiên thanh toán trước các chi phí khác.  \n## 4) Việc đóng Bảo hiểm xã hội được tính căn cứ vào tiền lương Cấp bậc (không tính phụ cấp liên quan đến đời sống)  \n## 5) Việc thay đổi ngày trả lương do trường hợp bất khả kháng phải thông báo cho người lao động trước 7 ngày. Công ty có trách nhiệm trả lại cho người lao động theo quy định của ngân hàng nếu chậm trả lương.  \n## 6) Khấu trừ lương  \nTrường hợp vắng mặt (do nghỉ không lương, ng

In [9]:
part = response.json()['choices'][0]['message']['content']

In [10]:
print(part)

# hợp sau đây:
a. Thuế thu nhập cá nhân  
b. Bảo hiểm xã hội, bảo hiểm y tế, bảo hiểm thất nghiệp  
c. Các khoản trừ trước  
d. Bồi thường  
d. Các chi phí khác do Công ty và Công đoàn thoả thuận.  

## 2) Trường hợp ngày thanh toán lương là ngày nghỉ lễ thì việc trả lương được thực hiện vào ngày làm việc trước đó.  
## 3) Tiền lương được ưu tiên thanh toán trước các chi phí khác.  
## 4) Việc đóng Bảo hiểm xã hội được tính căn cứ vào tiền lương Cấp bậc (không tính phụ cấp liên quan đến đời sống)  
## 5) Việc thay đổi ngày trả lương do trường hợp bất khả kháng phải thông báo cho người lao động trước 7 ngày. Công ty có trách nhiệm trả lại cho người lao động theo quy định của ngân hàng nếu chậm trả lương.  
## 6) Khấu trừ lương  
Trường hợp vắng mặt (do nghỉ không lương, nghi hưởng chế độ Bảo hiểm xã hội, ...), công ty sẽ khấu trừ vào lương của người lao động theo công thức sau:  
$$ \text{Khấu trừ 1 ngày vắng mặt} = \frac{\text{Lương thường xuyên}}{\text{Số ngày làm việc tiêu chuẩn hàng

In [11]:
with open(f"parsing_result/customer.md", "a", encoding="utf-8") as file:
    file.write(part + "\n\n\n")

In [6]:
import re

# Sample text containing tables
text = """
Table 1:
STT | Tên tổ chức, cá nhân nước ngoài | Địa chỉ | Đại học trở lên | Cao đẳng | Trung cấp | Sơ cấp nghề | Chưa qua đào tạo
---|---------------------------------|----------|----------------|---------|-----------|-------------|-----------------
(1) | (2)                             | (3)      | (4)            | (5)     | (6)       | (7)         | (8)
1   |                                 |          |                |         |           |             |
... |                                 |          |                |         |           |             |

Table 2:
STT | Tên tổ chức, cá nhân nước ngoài | Địa chỉ | Số lao động đã được tổ chức có thẩm quyền tuyển, quản lý người lao động Việt Nam tuyển chọn, giới thiệu | Số lao động được tổ chức, cá nhân nước ngoài tuyển trực tiếp | Số lao động được tổ chức, cá nhân nước ngoài tuyển thông qua tổ chức dịch vụ việc làm và doanh nghiệp cho thuê lại lao động
     |                                |          | Lao động đã ký HĐLĐ chia theo trình độ chuyên môn kỹ thuật                                                                                          | Lao động đã ký HĐLĐ chia theo trình độ chuyên môn kỹ thuật                                                                              | Lao động đã ký HĐLĐ chia theo trình độ chuyên môn kỹ thuật    
     |                                |          | Tổng số | Đại học trở lên | Cao đẳng | Trung cấp | Sơ cấp nghề | Chưa qua đào tạo                                                                                   | Tổng số | Đại học trở lên | Cao đẳng | Trung cấp | Sơ cấp nghề | Chưa qua đào tạo
     |                                |          | (4)      | (5)              | (6)      | (7)       | (8)         | (9)                                                                                               | (10)     | (11)             | (12)      | (13)      | (14)        | (15)                                                                                                      | (16)     | (17)             | (18)      | (19)      | (20)        | (21)
(1)  | (2)                            | (3)      |          |                  |          |           |             |                                                                                                    |          |                  |           |           |             |                                                                                                      |          |                  |           |           |             |
1    |                                |          |          |                  |          |           |             |                                                                                                    |          |                  |           |           |             |
...  |                                |          |          |                  |          |           |             |                                                                                                    |          |                  |           |           |             |
"""

# Improved regex pattern to match both tables
table_pattern = r"Table \d+:\s*(.*?)(?=Table \d+:|$)"

# Find all tables
tables = re.findall(table_pattern, text, re.DOTALL)

# Print extracted tables
for table in tables:
    print(f"{table}")


STT | Tên tổ chức, cá nhân nước ngoài | Địa chỉ | Đại học trở lên | Cao đẳng | Trung cấp | Sơ cấp nghề | Chưa qua đào tạo
---|---------------------------------|----------|----------------|---------|-----------|-------------|-----------------
(1) | (2)                             | (3)      | (4)            | (5)     | (6)       | (7)         | (8)
1   |                                 |          |                |         |           |             |
... |                                 |          |                |         |           |             |


STT | Tên tổ chức, cá nhân nước ngoài | Địa chỉ | Số lao động đã được tổ chức có thẩm quyền tuyển, quản lý người lao động Việt Nam tuyển chọn, giới thiệu | Số lao động được tổ chức, cá nhân nước ngoài tuyển trực tiếp | Số lao động được tổ chức, cá nhân nước ngoài tuyển thông qua tổ chức dịch vụ việc làm và doanh nghiệp cho thuê lại lao động
     |                                |          | Lao động đã ký HĐLĐ chia theo trình độ chuyên 

In [55]:
def convert_latex_to_text(latex):
    # Remove LaTeX \text{} commands
    latex = re.sub(r'\\text\{(.*?)\}', r'\1', latex)

    # Replace LaTeX curly brackets \left\{ \right\} with normal curly brackets
    latex = latex.replace(r'\left\{', '{')
    latex = latex.replace(r'\right\}', '}')

    # Replace fractions \frac{}{}
    latex = re.sub(r'\\frac\{(.*?)\}\{(.*?)\}', r'(\1)/(\2)', latex)

    # Replace LaTeX brackets \left\[ \right] with normal brackets
    latex = re.sub(r'\\left\[', '[', latex)
    latex = re.sub(r'\\right\]', ']', latex)

    # Replace multiplication symbol \times
    latex = latex.replace(r'\times', '*')

    # Remove the `\,` LaTeX spacing and combine numbers with units (days, hours)
    latex = re.sub(r'(\d+)\s*\\,\s*\((.*?)\)', r'\1(\2)', latex)

    # Replace other LaTeX commands like \% to %
    latex = latex.replace(r'\%', '%')

    return latex

def to_raw_string(s):
    """
    Convert a regular string to a raw string representation.
    
    Args:
        s (str): The regular string to convert.
        
    Returns:
        str: The raw string representation.
    """
    # Replace single backslashes with double backslashes
    return s.replace("\\", "\\\\")

In [57]:
import re

# Sample text containing LaTeX expressions
text = """
Trường hợp vắng mặt (do nghỉ không lương, nghi hưởng chế độ Bảo hiểm xã hội, ...), công ty sẽ khấu trừ vào lương của người lao động theo công thức sau:  
$$ \text{Khấu trừ 1 ngày vắng mặt} = \frac{\text{Lương thường xuyên}}{\text{Số ngày làm việc tiêu chuẩn hàng tháng ( = 22 )}} $$  

# Điều 7. Trả lương trong thời gian thử việc  
Trong thời gian thử việc, tiền lương được tính theo công thức sau:  
$$ \text{Lương tháng trong thời gian thử việc} = \left[\text{Lương thường xuyên} \times 85\% \right] \\ = \frac{\text{Số ngày làm việc tiêu chuẩn hàng tháng (22 ngày)}}{\text{Ngày làm việc tiêu chuẩn trong tháng}} $$  

# Điều 8. Thanh toán đặc biệt  
Khi xem xét yêu cầu các trường hợp sau đây, Công ty phải trả lương cho người lao động theo phần công việc hoàn thành và không phụ thuộc vào Điều 6 - Ngày trả lương:  
1) Nhân viên/cán bộ chức.  
2) Chấm dứt hợp đồng lao động.  
3) Nhân viên/Cán bộ gia đình họ cần tiền để chi trả cho tai nạn, thương tích, bệnh tật hoặc thiên tai.  
4) Nhân viên/Cán bộ họ cần tiền để chi trả cho đám cưới, sinh con, tang lễ.
"""

# Regex to capture LaTeX expressions between $$
latex_pattern = r"\$\$(.*?)\$\$"

# Find all LaTeX expressions
latex_expressions = re.findall(latex_pattern, text, re.DOTALL)
s = "\text{Khấu trừ 1 ngày vắng mặt} = \frac{\text{Lương thường xuyên}}{\text{Số ngày làm việc tiêu chuẩn hàng tháng ( = 22 )}}"
print(r"{}".format(to_raw_string(s)))


	ext{Khấu trừ 1 ngày vắng mặt} = rac{	ext{Lương thường xuyên}}{	ext{Số ngày làm việc tiêu chuẩn hàng tháng ( = 22 )}}


In [44]:
import re

# List of LaTeX expressions (from previous regex extraction)
latex_expressions = [
    "\text{Khấu trừ 1 ngày vắng mặt} = \frac{\text{Lương thường xuyên}}{\text{Số ngày làm việc tiêu chuẩn hàng tháng ( = 22 )}} + 100",
    "\text{Lương tháng trong thời gian thử việc} = \left[\text{Lương thường xuyên} \times 85\% \right]  \\ = \frac{\text{Số ngày làm việc tiêu chuẩn hàng tháng (22 ngày)}}{\text{Ngày làm việc tiêu chuẩn trong tháng}}",
    """
\text{Tiền lương làm việc vào ban đêm} = \left\{ \text{Tiền lương giờ thực trả của công việc đang làm vào ngày làm việc bình thường} + \text{Tiền lương giờ thực trả của công việc đang làm vào ngày làm việc bình thường} \times \text{Mức ít nhất 30\%} \right\} \times \text{Số giờ làm việc vào ban đêm}
""",
    """\text{Phụ cấp làm thêm ngoài giờ (mỗi 1 giờ)} = \frac{(\text{Lương cấp bậc}) \times \text{Hệ số phụ cấp}^{*1)}}{\text{Ngày làm việc tiêu chuẩn hàng tháng} \times \text{Số giờ làm việc tiêu chuẩn mỗi ngày}}
= \frac{(\text{Lương cấp bậc}) \times \text{Hệ số phụ cấp}^{*1)}}{22 \, (\text{ngày}) \times 8 \, (\text{giờ})}"""
]

# Function to convert LaTeX expression to normal text
def convert_latex_to_text(latex):
    # Remove LaTeX \text{} commands
    latex = re.sub(r'\\text\{(.*?)\}', r'\1', latex)

    # Replace LaTeX curly brackets \left\{ \right\} with normal curly brackets
    latex = latex.replace(r'\left\{', '{')
    latex = latex.replace(r'\right\}', '}')

    # Replace fractions \frac{}{}
    latex = re.sub(r'\\frac\{(.*?)\}\{(.*?)\}', r'(\1)/(\2)', latex)

    # Replace LaTeX brackets \left\[ \right] with normal brackets
    latex = re.sub(r'\\left\[', '[', latex)
    latex = re.sub(r'\\right\]', ']', latex)

    # Replace multiplication symbol \times
    latex = latex.replace(r'\times', '*')

    # Remove the `\,` LaTeX spacing and combine numbers with units (days, hours)
    latex = re.sub(r'(\d+)\s*\\,\s*\((.*?)\)', r'\1(\2)', latex)

    # Replace other LaTeX commands like \% to %
    latex = latex.replace(r'\%', '%')

    return latex
# Convert and print each LaTeX expression

# for expression in latex_expressions:
#     # r_ex = r"{}".format(expression)
#     print(r_ex)
#     normal_expression = convert_latex_to_text(r_ex)
#     # print(f"Expression (Normal Form):\n{normal_expression}\n")


In [45]:
regular_string = "\text{Khấu trừ 1 ngày vắng mặt} = \frac{\text{Lương thường xuyên}}{\text{Số ngày làm việc tiêu chuẩn hàng tháng ( = 22 )}} + 100"
s = r"""{}""".format(latex_expressions[1])
convert_latex_to_text(s)

'\text{Lương tháng trong thời gian thử việc} = [\text{Lương thường xuyên} \times 85% \right]  \\ = \x0crac{\text{Số ngày làm việc tiêu chuẩn hàng tháng (22 ngày)}}{\text{Ngày làm việc tiêu chuẩn trong tháng}}'

In [29]:
import re

# LaTeX expression example
latex_expressions = [
    r"\text{Tiền lương làm việc vào ban đêm} = \left\{ \text{Tiền lương giờ thực trả của công việc đang làm vào ngày làm việc bình thường} + \text{Tiền lương giờ thực trả của công việc đang làm vào ngày làm việc bình thường} \times \text{Mức ít nhất 30\%} \right\} \times \text{Số giờ làm việc vào ban đêm}"
]

# Function to convert LaTeX expression to normal text
def convert_latex_to_text(latex):
    # Remove LaTeX \text{} commands
    latex = re.sub(r'\\text\{(.*?)\}', r'\1', latex)

    # Replace LaTeX curly brackets \left\{ \right\} with normal curly brackets
    latex = latex.replace(r'\left\{', '{')
    latex = latex.replace(r'\right\}', '}')

    # Replace fractions \frac{}{}
    latex = re.sub(r'\\frac\{(.*?)\}\{(.*?)\}', r'(\1)/(\2)', latex)

    # Replace LaTeX brackets \left\[ \right] with normal brackets
    latex = re.sub(r'\\left\[', '[', latex)
    latex = re.sub(r'\\right\]', ']', latex)

    # Replace multiplication symbol \times
    latex = latex.replace(r'\times', '*')

    # Remove the `\,` LaTeX spacing and combine numbers with units (days, hours)
    latex = re.sub(r'(\d+)\s*\\,\s*\((.*?)\)', r'\1(\2)', latex)

    # Replace other LaTeX commands like \% to %
    latex = latex.replace(r'\%', '%')

    return latex

# Convert and print each LaTeX expression
for i, expression in enumerate(latex_expressions, start=1):
    normal_expression = convert_latex_to_text(expression)
    print(f"Expression {i} (Normal Form):\n{normal_expression}\n")


Expression 1 (Normal Form):
Tiền lương làm việc vào ban đêm = { Tiền lương giờ thực trả của công việc đang làm vào ngày làm việc bình thường + Tiền lương giờ thực trả của công việc đang làm vào ngày làm việc bình thường * Mức ít nhất 30% } * Số giờ làm việc vào ban đêm



In [106]:
import re

# Đoạn văn bản chứa biểu thức LaTeX
text = r"""
Trường hợp vắng mặt (do nghỉ không lương, nghi hưởng chế độ Bảo hiểm xã hội, ...), công ty sẽ khấu trừ vào lương của người lao động theo công thức sau:  
$$ \text{Khấu trừ 1 ngày vắng mặt} = \frac{\text{Lương thường xuyên}}{\text{Số ngày làm việc tiêu chuẩn hàng tháng ( = 22 )}} $$  

# Điều 7. Trả lương trong thời gian thử việc  
Trong thời gian thử việc, tiền lương được tính theo công thức sau:  
$$ \text{Lương tháng trong thời gian thử việc} = \left[\text{Lương thường xuyên} \times 85\% \right] \\ = \frac{\text{Số ngày làm việc tiêu chuẩn hàng tháng (22 ngày)}}{\text{Ngày làm việc tiêu chuẩn trong tháng}} $$  

# Điều 8. Thanh toán đặc biệt  
Khi xem xét yêu cầu các trường hợp sau đây, Công ty phải trả lương cho người lao động theo phần công việc hoàn thành và không phụ thuộc vào Điều 6 - Ngày trả lương:  
1) Nhân viên/cán bộ chức.  
2) Chấm dứt hợp đồng lao động.  
3) Nhân viên/Cán bộ gia đình họ cần tiền để chi trả cho tai nạn, thương tích, bệnh tật hoặc thiên tai.  
4) Nhân viên/Cán bộ họ cần tiền để chi trả cho đám cưới, sinh con, tang lễ.
"""
raw_text = repr(text)

# Regex để lấy tất cả các biểu thức LaTeX nằm giữa hai cặp dấu $$
latex_patterns = re.findall(r'\$\$([\s\S]*?)\$\$', raw_text)

# In kết quả
print("Các biểu thức LaTeX:")
for pattern in latex_patterns:
    print(pattern)


Các biểu thức LaTeX:
 \\text{Khấu trừ 1 ngày vắng mặt} = \\frac{\\text{Lương thường xuyên}}{\\text{Số ngày làm việc tiêu chuẩn hàng tháng ( = 22 )}} 
 \\text{Lương tháng trong thời gian thử việc} = \\left[\\text{Lương thường xuyên} \\times 85\\% \\right] \\\\ = \\frac{\\text{Số ngày làm việc tiêu chuẩn hàng tháng (22 ngày)}}{\\text{Ngày làm việc tiêu chuẩn trong tháng}} 


In [107]:
def convert_latex_to_text(latex):
    # Remove LaTeX \text{} commands
    latex = re.sub(r'\\text\{(.*?)\}', r'\1', latex)

    # Replace LaTeX curly brackets \left\{ \right\} with normal curly brackets
    latex = latex.replace(r'\left\{', '{')
    latex = latex.replace(r'\right\}', '}')

    # Replace fractions \frac{}{}
    latex = re.sub(r'\\frac\{(.*?)\}\{(.*?)\}', r'(\1)/(\2)', latex)

    # Replace LaTeX brackets \left\[ \right] with normal brackets
    latex = re.sub(r'\\left\[', '[', latex)
    latex = re.sub(r'\\right\]', ']', latex)

    # Replace multiplication symbol \times
    latex = latex.replace(r'\times', '*')

    # Remove the `\,` LaTeX spacing and combine numbers with units (days, hours)
    latex = re.sub(r'(\d+)\s*\\,\s*\((.*?)\)', r'\1(\2)', latex)

    # Replace other LaTeX commands like \% to %
    latex = latex.replace(r'\%', '%').replace('\\', ' ').strip()

    return latex
# Chuyển đổi tất cả các biểu thức LaTeX
converted_texts = [convert_latex_to_text(pattern) for pattern in latex_patterns]

# In kết quả chuyển đổi
print("\nCác biểu thức bình thường:")
for text in converted_texts:
    print(text)



Các biểu thức bình thường:
Khấu trừ 1 ngày vắng mặt =  ( Lương thường xuyên)/( Số ngày làm việc tiêu chuẩn hàng tháng ( = 22 ))
Lương tháng trong thời gian thử việc =  [ Lương thường xuyên  * 85 %  ]      =  ( Số ngày làm việc tiêu chuẩn hàng tháng (22 ngày))/( Ngày làm việc tiêu chuẩn trong tháng)


In [22]:
prompt_system = """You are an AI assistant that processes images containing legal content. Your task is to transcribe the content from the image into markdown format with the following rules:
        
                        1. Use `#` for titles if you see the name of a Law or Decree.
                        2. Use `##` if you see the word "Chương" or "Mẫu số" at the beginning of a sentence.
                        3. Use `###` if you see the word "Mục" at the beginning of a sentence.
                        4. Use `####` if you see the word "Điều" at the beginning of a sentence.
                        5. Do not use any '#' for a), b), c),... etc or 1), 2), 3),...etc
                        6. If you encounter a equation, notice the following rules:
                            - The equation can be on multiple lines, detect all the lines of the equation.
                             - Convert the method in the image into a single latex block with all expressions inside a pair of dollar signs ($$). Try to get both legs before and after the equal sign (=) (i.e. $$ \text{Gross bonus amount} = \text{Bonus} \times \left(100 - \frac{\text{Date of absent in a year}}{22 \times 12}\right) $$)
                        7. Only respond with the markdown content, without any additional explanation or description.
                        8. Don't annotate ```markdown\n  in the response
                    """
detect_table_prompt = """You are an expert in table dectection. If the input image have any tables,focus on each table and response the markdown of each table with the index of each table in the begin of table (i.e. Table 1, Table 2, ...etc). Try to keep all the text of each table at the right cell in the image and do not annotate ```markdown. If the image don't have any tables, response "0" """
api_key = os.environ["OPENAI_API_KEY"]


In [23]:
import fitz
from PIL import Image
import io
import base64
from io import BytesIO
import requests
import os
import time
import re
def pdf_to_images(pdf_path, zoom_x=2.0, zoom_y=2.0):
    pdf_document = fitz.open(pdf_path)
    images = []
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        mat = fitz.Matrix(zoom_x, zoom_y)
        pix = page.get_pixmap(matrix=mat)
        img = Image.open(io.BytesIO(pix.tobytes("png")))
        images.append(img)
    image_base64s = []

    for image in images:
        buffered = BytesIO()
        image.save(buffered, format="PNG")
        image_bytes = buffered.getvalue()
        image_base64s.append(base64.b64encode(image_bytes).decode("utf-8"))
    return image_base64s

In [24]:
image_base64s = pdf_to_images("thuvienphapluat/law_2. Tai lieu Khach hang cung cap_5. Salary Regulation 2024 (V) (1).pdf")

In [25]:
def detect_table(image_base64):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
            "role": "system",
            "content": [
                {
                "type": "text",
                "text": detect_table_prompt
                },
            ]
            },
            {
            "role": "user",
            "content": [
                {
                "type": "text",
                "text": """Detect table in this image"""
                },
                {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{image_base64}"
                }
                }
            ]
            }
        ],
        "max_tokens": 4096
    }
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response.json()['choices'][0]['message']['content']

In [26]:
def concatenate_pages(pages, file_name):
    pdf_content = ""
    for page in pages:
        i = page["page"]
        page_content = f"#Trang {i}\n\n" + page["content"] + f"\n\n"
        pdf_content += page_content
    pdf_info = {"content": pdf_content, "reference": pages[0]["reference"], "src": file_name}
    return pdf_info

In [27]:
def parse_pdf(image_base64s, file_name):
    pages = []
    tables = []
    law_name = ""
    for i in range(0, len(image_base64s)):
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {api_key}"
        }

        payload = {
            "model": "gpt-4o-mini",
            "messages": [
                {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": prompt_system
                    },
                ]
                },
                {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": "Transcribe the content from this image into markdown format"
                    },
                    {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image_base64s[i]}"
                    }
                    }
                ]
                }
            ],
            "max_tokens": 4096
        }

        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        page_content = response.json()['choices'][0]['message']['content']
        print(f"{i+1}:\n{page_content}")
        if i == 0:
            pattern = [r'(BỘ LUẬT\s*\n)(.*)', r'(LUẬT\s*\n)(.*)', r'(NGHỊ ĐỊNH\s*\n)(.*)', r'(QUY ĐỊNH\s*.*\n)(.*)']
            text = ""
            for p in pattern:
                text = re.findall(p, page_content)   
                if len(text) > 0:
                    law_name = text[0][0].strip() + " " + text[0][1].strip()
                    break

        pages.append({"page": i+1, "content": page_content, "reference": law_name})
        num_of_tables = detect_table(image_base64s[i])
        if num_of_tables != "No":
            print(num_of_tables)
            tables.append({"page": i+1, "content": num_of_tables, "reference": law_name, "src": file_name})            
        time.sleep(2)
    
    pdf_info = concatenate_pages(pages=pages, file_name=file_name)
    return pdf_info, tables

In [28]:
pdf_info, tables = parse_pdf(image_base64s, "law_2. Tai lieu Khach hang cung cap_5. Salary Regulation 2024 (V) (1).pdf")

1:
# QUY ĐỊNH VỀ TIỀN LƯƠNG – TIỀN THƯỞNG

Sửa đổi 1: Ngày 01 tháng 01 năm 2011  
Sửa đổi 2: Ngày 01 tháng 05 năm 2011  
Sửa đổi 3: Ngày 01 tháng 01 năm 2012  
Sửa đổi 4: Ngày 01 tháng 05 năm 2013  
Sửa đổi 5: Ngày 01 tháng 01 năm 2014  
Sửa đổi 6: Ngày 01 tháng 01 năm 2015  
Sửa đổi 7: Ngày 01 tháng 01 năm 2016  
Sửa đổi 8: Ngày 01 tháng 01 năm 2017  
Sửa đổi 9: Ngày 01 tháng 01 năm 2022  
Sửa đổi 10: Ngày 01 tháng 01 năm 2023  
Sửa đổi 11: Ngày 01 tháng 12 năm 2023  

# CÔNG TY TNHH BƠM Ebara VIỆT NAM

Phê duyệt bởi  
Tổng giám đốc  

Soạn thảo bởi  
Ban Nhân sự  

Tetsuya Fuchida  
Nguyen Tu Khanh
2:
# Điều 1: Quy định về tiền lương
Công ty sẽ trả lương cho cán bộ công nhân viên theo lương tháng và lương ngày, coi đó là để bảo đảm sức lao động trên cơ sở xem xét kết quả trình làm việc, thành tích, năng suất, chất lượng, hiệu quả công việc và sinh hoạt phí hợp với tình hình xã hội và kết quả kinh doanh của công ty.

# Điều 2: Cơ cấu tiền lương
Lương thường xuyên (Theo Phụ lục I)

(*)

In [31]:
pdf_info

{'content': '#Trang 1\n\n# QUY ĐỊNH VỀ TIỀN LƯƠNG – TIỀN THƯỞNG\n\nSửa đổi 1: Ngày 01 tháng 01 năm 2011  \nSửa đổi 2: Ngày 01 tháng 05 năm 2011  \nSửa đổi 3: Ngày 01 tháng 01 năm 2012  \nSửa đổi 4: Ngày 01 tháng 05 năm 2013  \nSửa đổi 5: Ngày 01 tháng 01 năm 2014  \nSửa đổi 6: Ngày 01 tháng 01 năm 2015  \nSửa đổi 7: Ngày 01 tháng 01 năm 2016  \nSửa đổi 8: Ngày 01 tháng 01 năm 2017  \nSửa đổi 9: Ngày 01 tháng 01 năm 2022  \nSửa đổi 10: Ngày 01 tháng 01 năm 2023  \nSửa đổi 11: Ngày 01 tháng 12 năm 2023  \n\n# CÔNG TY TNHH BƠM Ebara VIỆT NAM\n\nPhê duyệt bởi  \nTổng giám đốc  \n\nSoạn thảo bởi  \nBan Nhân sự  \n\nTetsuya Fuchida  \nNguyen Tu Khanh\n\n#Trang 2\n\n# Điều 1: Quy định về tiền lương\nCông ty sẽ trả lương cho cán bộ công nhân viên theo lương tháng và lương ngày, coi đó là để bảo đảm sức lao động trên cơ sở xem xét kết quả trình làm việc, thành tích, năng suất, chất lượng, hiệu quả công việc và sinh hoạt phí hợp với tình hình xã hội và kết quả kinh doanh của công ty.\n\n# Điều 2

In [65]:
pdf_info["content"] = pdf_info["content"].replace("*Trang", "* Trang")
pdf_info


{'content': '* Trang 1\n\n# QUY ĐỊNH VỀ TIỀN LƯƠNG – TIỀN THƯỞNG\n\nSửa đổi 1: Ngày 01 tháng 01 năm 2011  \nSửa đổi 2: Ngày 01 tháng 05 năm 2011  \nSửa đổi 3: Ngày 01 tháng 01 năm 2012  \nSửa đổi 4: Ngày 01 tháng 05 năm 2013  \nSửa đổi 5: Ngày 01 tháng 01 năm 2014  \nSửa đổi 6: Ngày 01 tháng 01 năm 2015  \nSửa đổi 7: Ngày 01 tháng 01 năm 2016  \nSửa đổi 8: Ngày 01 tháng 01 năm 2017  \nSửa đổi 9: Ngày 01 tháng 01 năm 2022  \nSửa đổi 10: Ngày 01 tháng 01 năm 2023  \nSửa đổi 11: Ngày 01 tháng 12 năm 2023  \n\n# CÔNG TY TNHH BƠM Ebara VIỆT NAM\n\nPhê duyệt bởi  \nTổng giám đốc  \n\nSoạn thảo bởi  \nBan Nhân sự  \n\nTetsuya Fuchida  \nNguyen Tu Khanh\n\n* Trang 2\n\n# Điều 1: Quy định về tiền lương\nCông ty sẽ trả lương cho cán bộ công nhân viên theo lương tháng và lương ngày, coi đó là để bảo đảm sức lao động trên cơ sở xem xét kết quả trình làm việc, thành tích, năng suất, chất lượng, hiệu quả công việc và sinh hoạt phí hợp với tình hình xã hội và kết quả kinh doanh của công ty.\n\n# Điều

In [71]:
with open(f"thuvienphapluat/parsed_file/new.md", "w", encoding="utf-8") as file:
    file.write(pdf_info["content"])

In [30]:
tables

[{'page': 5,
  'content': 'Table 1\n\n|                                 | Lương thường xuyên |\n|------------------------------|-------------------|\n| Khấu trừ 1 ngày vắng mặt   | =                 |\n| =                           | Số ngày làm việc tiêu chuẩn hàng tháng (= 22*) |\n\nTable 2\n\n|                                                        |                 |\n|--------------------------------------------------------|-----------------|\n| Lương tháng trong thời gian thử việc                 |                 |\n| [Lương thường xuyên * 85% ]                            | =               |\n| =                                                      | Số ngày làm việc |\n| Ngày làm việc tiêu chuẩn hàng tháng (22 ngày)         | *               |\n| *                                                      | trong tháng     |',
  'reference': 'QUY ĐỊNH VỀ TIỀN LƯƠNG – TIỀN THƯỞNG ',
  'src': 'law_2. Tai lieu Khach hang cung cap_5. Salary Regulation 2024 (V) (1).pdf'},
 {'page': 6,
  

In [32]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
import re
def get_chunk_summary(content) -> str:
    PROMPT = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """
                Bạn là người quản lý một nhóm các đoạn văn bản, bảng hoặc biểu đồ, trong đó mỗi đoạn đều đại diện cho một nhóm các câu nói về một chủ đề tương tự nhau.
                Bạn cần tạo ra một tóm tắt ngắn gọn trong 1 câu để thông báo cho người xem biết đoạn văn bản, bảng hoặc biểu đó nói về chủ đề gì.

                Một tóm tắt tốt sẽ nói lên nội dung của đoạn văn bản, bảng hoặc biểu đồ

                Bạn sẽ nhận được một đề xuất mới sẽ được thêm vào đoạn văn bản, bảng hoặc biểu mới. Đoạn văn bản, bảng hoặc biểu đồ mới này cần có một tóm tắt.

                Tóm tắt của bạn nên có tính khái quát tương đối. Nếu bạn nhận được một đề xuất về táo, hãy khái quát nó thành thực phẩm.
                Hoặc nếu là tháng, hãy khái quát nó thành "ngày và thời gian".

                Ví dụ:
                Đầu vào: Đề xuất: Greg thích ăn pizza
                Đầu ra: Đoạn văn bản này chứa thông tin về các loại thực phẩm mà Greg thích ăn.

                Chỉ phản hồi lại bằng tóm tắt của đoạn văn bản mới, không thêm gì khác.
                """,
            ),
            ("user", "Xác định tóm tắt của đoạn văn bản mới mà đề xuất này sẽ được thêm vào:\n{content}"),
        ]
    )
    llm = ChatOpenAI(model='gpt-4o-mini', openai_api_key=api_key, temperature=0)

    runnable = PROMPT | llm

    new_chunk_summary = runnable.invoke({
        "content": content
    }).content

    return new_chunk_summary

In [35]:
def convert_latex_to_text(latex):
    latex = re.sub(r'\\text\{(.*?)\}', r'\1', latex)

    latex = latex.replace(r'\left\{', '{')
    latex = latex.replace(r'\right\}', '}')

    latex = re.sub(r'\\frac\{(.*?)\}\{(.*?)\}', r'(\1)/(\2)', latex)

    latex = re.sub(r'\\left\[', '[', latex)
    latex = re.sub(r'\\right\]', ']', latex)

    latex = latex.replace(r'\times', '*')
    latex = latex.replace(r'\div', '/')

    latex = re.sub(r'(\d+)\s*\\,\s*\((.*?)\)', r'\1(\2)', latex)

    latex = latex.replace(r'\%', '%').replace('\\', ' ').strip()

    return latex

In [56]:
latex = r"""\\text{Khấu trừ 1 ngày vắng mặt} = \\text{Lương thường xuyên} \\div \\text{Số ngày làm việc tiêu chuẩn hàng tháng (= 22*)}"""
convert_latex_to_text(latex=latex)

'Khấu trừ 1 ngày vắng mặt =  Lương thường xuyên  /  Số ngày làm việc tiêu chuẩn hàng tháng (= 22*)'

In [66]:
def create_article(pdf_info):
    muc_luc = list()
    pattern = r'(\* Trang \d+)(.*?)(?=\* Trang \d+|$)'
    article_pattern = r"#+\s+Điều\s+\d+[:.]\s+[^\n]+"
    matches = re.findall(pattern, pdf_info["content"], re.DOTALL)
    pages = [match[0] + match[1] for match in matches]
    print(len(pages))
    for i, page in enumerate(pages, 1):
    
        a_matches = re.findall(article_pattern, page)
        for a in a_matches:
            muc_luc.append({"object": a, "page": i})
    

    documents = []
    chapter_pattern = r'(#+\s+Chương\s+[IVXLCDM]+.*?)(?=#+\s+Chương\s+[IVXLCDM]+|$)'


    chapters = re.findall(chapter_pattern, pdf_info["content"], re.S)

    if len(chapters) == 0:
        article_pattern = r'(#+\s+Điều\s+\d+[:.].*?)(?=\n+#+\s+Mẫu số|\n+#+\s+Phụ lục|\n+#+\s+Điều\s+\d+|\n+#+\s+Chương\s+[IVXLCDM]+|$)'
        articles = re.findall(article_pattern, pdf_info["content"], re.DOTALL)
        for article in articles:
            latex_patterns = r'\$\$([\s\S]*?)\$\$'
            latex_matches_origin = re.findall(latex_patterns, article)
            latex_matches = re.findall(latex_patterns, repr(article))
            for i in range(len(latex_matches)):
                clean_latex = convert_latex_to_text(latex=latex_matches[i])
                print(clean_latex)
                article.replace(latex_matches[i], clean_latex)

            a_content = re.sub(r'\* Trang \d+', '', article)
            a_content = re.sub(r'[^\w\s.,;:|]', '', a_content)
            a_content = re.sub(r'\n+', ' ', a_content).strip()
            for muc in muc_luc:
                if article.find(muc["object"]) != -1:
                    documents.append({"raw_text": article, "content": a_content, "summary": get_chunk_summary(a_content), "src": pdf_info["src"], "chapter": "", "page": muc["page"], "reference": pdf_info["reference"]})
                    break
    else:
        article_pattern = r'(#+\s+Điều\s+\d+[:.].*?)(?=\n+#+\s+Mẫu số|\n+#+\s+Phụ lục|\n+#+\s+Điều\s+\d+|\n+#+\s+Chương\s+[IVXLCDM]+|$)'
        chapter_to_articles = {}
        for chapter in chapters:
            chapter_title_match = re.search(r'##\s+(Chương\s+\w+)', chapter)
            if chapter_title_match:
                chapter_title = chapter_title_match.group(1)
                chapter_to_articles[chapter_title] = []
                
                articles = re.findall(article_pattern, chapter, re.S)
                for article in articles:
                    latex_patterns = r'\$\$([\s\S]*?)\$\$'
                    latex_matches_origin = re.findall(latex_patterns, article)
                    latex_matches = re.findall(latex_patterns, repr(article))
                    for i in range(len(latex_matches)):
                        clean_latex = convert_latex_to_text(latex=latex_matches[i])
                        print(clean_latex)
                        article.replace(latex_matches[i], clean_latex)
                        
                    a_content = re.sub(r'\* Trang \d+', '', article)
                    a_content = re.sub(r'[^\w\s.,;:|]', '', a_content)
                    a_content = re.sub(r'\n+', ' ', a_content).strip()
                    for muc in muc_luc:
                        if article.find(muc["object"]) != -1:
                            documents.append({"raw_text": article, "content": a_content, "summary": get_chunk_summary(a_content), "src": pdf_info["src"], "chapter": chapter_title, "page": muc["page"], "reference": pdf_info["reference"]})
                            break
    return documents

In [67]:
documents = create_article(pdf_info=pdf_info)

10
Khấu trừ 1 ngày vắng mặt =  Lương thường xuyên  /  Số ngày làm việc tiêu chuẩn hàng tháng (= 22*)
Lương tháng trong thời gian thử việc = [ Lương thường xuyên  * 85 %]  *  Số ngày làm việc tiêu chuẩn hàng tháng (22 ngày)
n= (Lương cấp bậc) * Hệ số phụ cấp^*1) n
n= (Lương cấp bậc) * Hệ số phụ cấp^*1)  n= (Lương cấp bậc) * Hệ số phụ cấp^*1)  n22 (ngày) * 8 (giờ) n
n= (Lương cấp bậc) * 0.3 n
n= (Lương cấp bậc) * 0.3  n= (Lương cấp bậc) * 0.3  n22 (ngày) * 8 (giờ) n
nLương cấp bậc =  ( Số ngày làm việc tiêu chuẩn hàng tháng (= 22))/( Thanh toán một ngày phép tồn) n
nTổng tiền thưởng = Tiền thưởng  * Tỷ lệ làm việc trong năm n
Tiền   , thưởng = Ngày   , vắng   , mặt   , trong   , năm  * (100 -  (22  * 12)/(Ngày   , vắng   , mặt   , trong   , năm))


In [64]:
documents

[{'raw_text': '# Điều 1: Quy định về tiền lương\nCông ty sẽ trả lương cho cán bộ công nhân viên theo lương tháng và lương ngày, coi đó là để bảo đảm sức lao động trên cơ sở xem xét kết quả trình làm việc, thành tích, năng suất, chất lượng, hiệu quả công việc và sinh hoạt phí hợp với tình hình xã hội và kết quả kinh doanh của công ty.',
  'content': 'Điều 1: Quy định về tiền lương Công ty sẽ trả lương cho cán bộ công nhân viên theo lương tháng và lương ngày, coi đó là để bảo đảm sức lao động trên cơ sở xem xét kết quả trình làm việc, thành tích, năng suất, chất lượng, hiệu quả công việc và sinh hoạt phí hợp với tình hình xã hội và kết quả kinh doanh của công ty.',
  'summary': 'Đoạn văn bản này chứa thông tin về quy định tiền lương cho cán bộ công nhân viên trong công ty.',
  'src': 'law_2. Tai lieu Khach hang cung cap_5. Salary Regulation 2024 (V) (1).pdf',
  'chapter': '',
  'page': 2,
  'reference': 'QUY ĐỊNH VỀ TIỀN LƯƠNG – TIỀN THƯỞNG '},
 {'raw_text': '# Điều 2: Cơ cấu tiền lương\

In [84]:
def create_table(tables):
    documents = []
    table_pattern = r"Table \d+\s*(.*?)(?=Table \d+|$)"
    for table_info in tables:
        tbls = re.findall(table_pattern, table_info["content"], re.DOTALL)
        for tbl in tbls:
            documents.append({"raw_text": tbl, "content": tbl, "summary": get_chunk_summary(tbl), "src": table_info["src"], "chapter": "", "page": table_info["page"], "reference": table_info["reference"]})

    return documents

In [85]:
documents = create_table(tables=tables)

In [86]:
len(documents)

7

In [ ]:
documents

In [87]:
table_pattern = r"Table \d+\s*(.*?)(?=Table \d+|$)"
tbls = re.findall(table_pattern, tables[1]["content"], re.DOTALL)

In [96]:
print(tables[5]["content"])

Table 1

| Number of years working at EVPC | Bonus amount (VND) |
|--------------------------------|---------------------|
| 5 years                        | 1.500.000           |
| 10 years                       | 3.000.000           |
| 15 years                       | 4.500.000           |
| 20 years                       | 6.000.000           |
| 25 years                       | 7.500.000           |
| 30 years                       | 9.000.000           |
| 35 năm                         | 10.500.000          |
| 40 năm                         | 12.000.000          |


In [88]:
len(tbls)

1

In [101]:
def create_equation(pdf_info):
    documents = []
    latex_patterns = r'\$\$([\s\S]*?)\$\$'
    pattern = r'(\* Trang \d+)(.*?)(?=\* Trang \d+|$)'
    matches = re.findall(pattern, pdf_info["content"], re.DOTALL)
    pages = [match[0] + match[1] for match in matches]
    for i, page in enumerate(pages, 1):
        raw_page = repr(page)
        latex_matches = re.findall(latex_patterns, raw_page)
        for latex in latex_matches:
            clean_latex = convert_latex_to_text(latex=latex)
            documents.append({"raw_text": latex, "content": clean_latex, "summary": get_chunk_summary(clean_latex), "src": pdf_info["src"], "chapter": "", "page": i, "reference": pdf_info["reference"]})

    return documents
        

In [102]:
documents = create_equation(pdf_info=pdf_info)

In [103]:
documents

[{'raw_text': ' \\\\text{Khấu trừ 1 ngày vắng mặt} = \\\\text{Lương thường xuyên} \\\\div \\\\text{Số ngày làm việc tiêu chuẩn hàng tháng (= 22*)} ',
  'content': 'Khấu trừ 1 ngày vắng mặt =  Lương thường xuyên  /  Số ngày làm việc tiêu chuẩn hàng tháng (= 22*)',
  'summary': 'Đoạn văn bản này chứa thông tin về cách tính khấu trừ lương cho ngày vắng mặt trong công việc.',
  'src': 'law_2. Tai lieu Khach hang cung cap_5. Salary Regulation 2024 (V) (1).pdf',
  'chapter': '',
  'page': 5,
  'reference': 'QUY ĐỊNH VỀ TIỀN LƯƠNG – TIỀN THƯỞNG '},
 {'raw_text': ' \\\\text{Lương tháng trong thời gian thử việc} = [\\\\text{Lương thường xuyên} \\\\times 85\\\\%] \\\\times \\\\text{Số ngày làm việc tiêu chuẩn hàng tháng (22 ngày)} ',
  'content': 'Lương tháng trong thời gian thử việc = [ Lương thường xuyên  * 85 %]  *  Số ngày làm việc tiêu chuẩn hàng tháng (22 ngày)',
  'summary': 'Đoạn văn bản này chứa thông tin về cách tính lương tháng trong thời gian thử việc.',
  'src': 'law_2. Tai lieu Kha